In this notebook, a pre-saved urbanaccess network into a h5 file is loaded and transformed into a pandana network. Each step is explained.

1. Load urbanaccess network and check column names and format

In [27]:
import urbanaccess as ua

# Define the file path to your saved network (by default the file is supposed to be located in the data folder)
h5_file_path = "transit_0820.h5"

# Load the network
network = ua.network.load_network(filename=h5_file_path)

Successfully read store: data/h5_files/transit_0820.h5 with the following keys: ['/edges', '/impedance_names', '/nodes', '/two_way']
Successfully read store: data/h5_files/transit_0820.h5 with the following keys: ['/edges', '/impedance_names', '/nodes', '/two_way']


In [28]:
# Print all available attributes
print(dir(network))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'net_connector_edges', 'net_edges', 'net_nodes', 'osm_edges', 'osm_nodes', 'transit_edges', 'transit_nodes']


In [29]:
print(network.net_nodes.head())  # Check first rows of nodes
print(network.net_edges.head())  # Check first rows of edges

               x          y
id_int                     
1      -1.976704  43.317669
2      -2.318462  43.088273
3      -1.800192  43.339582
4      -1.976704  43.317669
5      -2.078908  43.135405
   from    to  weight
0     1  5087    45.0
1     2  5088    55.0
2     3     4    22.0
3     4     5    32.0
4     5  5089    28.0


2. Load the nodes and edges into two separate dataframes to change format

In [30]:
node_df = network.net_nodes
print(node_df.head())

               x          y
id_int                     
1      -1.976704  43.317669
2      -2.318462  43.088273
3      -1.800192  43.339582
4      -1.976704  43.317669
5      -2.078908  43.135405


In [31]:
node_df.reset_index(inplace=True)

In [32]:
print(node_df.head())

   id_int         x          y
0       1 -1.976704  43.317669
1       2 -2.318462  43.088273
2       3 -1.800192  43.339582
3       4 -1.976704  43.317669
4       5 -2.078908  43.135405


In [33]:
edge_df = network.net_edges
print(edge_df.head())

   from    to  weight
0     1  5087    45.0
1     2  5088    55.0
2     3     4    22.0
3     4     5    32.0
4     5  5089    28.0


In [34]:
print(node_df.columns)

Index(['id_int', 'x', 'y'], dtype='object')


3. Perform the changes necessary to the nodes_df

In [35]:
import pandas as pd

# Merge using the updated 'id_int' column instead of 'id'
tmp = pd.merge(edge_df, node_df[['id_int']], left_on='from', right_index=True, sort=False, copy=False, how='left')

# Assign the 'from_int' column based on 'id_int' from node_df
tmp['from_int'] = tmp['id_int']

# Drop the unnecessary columns
tmp.drop(['id_int'], axis=1, inplace=True)

# Check the resulting dataframe
print(tmp.head())

   from    to  weight  from_int
0     1  5087    45.0       2.0
1     2  5088    55.0       3.0
2     3     4    22.0       4.0
3     4     5    32.0       5.0
4     5  5089    28.0       6.0


In [36]:
# Merge to get the 'to_int' column
tmp = pd.merge(tmp, node_df[['id_int']], left_on='to', right_index=True, sort=False, copy=False, how='left')

# Assign the 'to_int' column based on 'id_int' from node_df
tmp['to_int'] = tmp['id_int']

# Drop the unnecessary columns
tmp.drop(['id_int'], axis=1, inplace=True)

# Check the resulting dataframe
print(tmp.head())


   from    to  weight  from_int  to_int
0     1  5087    45.0       2.0  5088.0
1     2  5088    55.0       3.0  5089.0
2     3     4    22.0       4.0     5.0
3     4     5    32.0       5.0     6.0
4     5  5089    28.0       6.0  5090.0


In [37]:
node_df.rename(columns={'id_int': 'id'}, inplace=True)
print(node_df.head())

   id         x          y
0   1 -1.976704  43.317669
1   2 -2.318462  43.088273
2   3 -1.800192  43.339582
3   4 -1.976704  43.317669
4   5 -2.078908  43.135405


4. Create another urbanaccess network but with the pandana format

In [38]:
network_formated=ua.network._format_pandana_edges_nodes(edge_df, node_df)

Edge and node tables formatted for Pandana with integer node ids: id_int, to_int, and from_int. Took 0.36 seconds


In [39]:
print(network_formated)

(         from    to     weight  from_int  to_int
0           1  5087  45.000000         1    5087
1           2  5088  55.000000         2    5088
2           3     4  22.000000         3       4
3           4     5  32.000000         4       5
4           5  5089  28.000000         5    5089
...       ...   ...        ...       ...     ...
645397  65641  5203   7.808245     65641    5203
645398  65641  5203   7.808245     65641    5203
645399  65641  5203   7.808245     65641    5203
645400  65641  5203   7.808245     65641    5203
645401  65641  5203   7.808245     65641    5203

[645402 rows x 5 columns],             id         x          y
id_int                             
1            1 -1.976704  43.317669
2            2 -2.318462  43.088273
3            3 -1.800192  43.339582
4            4 -1.976704  43.317669
5            5 -2.078908  43.135405
...        ...       ...        ...
126263  126263 -2.211893  43.298687
126264  126264 -2.213562  43.296225
126265  126265 -2.21444

5. Load the nodes and edges of the formated network

In [40]:
edges_df, nodes_df = network_formated

6. Create a pandana network with the formated nodes and edges

In [41]:
import pandana

# Extract the 'from_int', 'to_int', and 'weight' columns for the edges
imp = pd.DataFrame(edges_df['weight'])

# Create a Pandana network
net = pandana.Network(nodes_df.x, nodes_df.y, 
                      edges_df.from_int, edges_df.to_int, imp, False)

7. Save the pandana network (in further uses, these two files can be loaded directly into a pandana network)

In [59]:
net.nodes_df.to_csv('nodes.csv')
net.edges_df.to_csv('edges.csv')

Use the network

Load some restaurant locations

Here we'll load the locations of restaurants listed on Open Street Map (using the same OSMnet extension as above), and then calculate some shortest paths between them.

In [43]:
# pip install osmnet
from pandana.loaders import osm

import warnings
warnings.filterwarnings('ignore')

In [44]:
restaurants = osm.node_query(
    42.89690109,-2.6032552,43.39526782,-1.72713422, tags='"amenity"="restaurant"')

Choose two at random:

In [55]:
res = restaurants.sample(2)
res

,lat,lon,amenity,name,note,addr:housenumber,addr:street,check_date,payment:mastercard,payment:visa,...,payment:nfc,survey:date,check_date:opening_hours,opening_hours:signed,name:de,diet:healthy,food,dog,payment:qr_code,addr:door
id,,,,,,,,,,,,,,,,,,,,,
3358548540,43.323538,-1.976657,restaurant,Ikaitz,NaN,21,Kolon pasealekua,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11743424965,43.119851,-2.406504,restaurant,Azpeitxi,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,leashed,no,NaN


Calculate the shortest route between them

In [56]:
nodes = net.get_node_ids(res.lon, res.lat).values
nodes

array([34037, 46533], dtype=int64)

Then get the routing between the nodes.

In [57]:
net.shortest_path(nodes[0], nodes[1])

array([ 34037,   8060,   8061,   8078,  75789,  60775,  31665,  31669,
       102036,  60773,  14790,  75749,  60769,  33631,   6629,  60763,
        31667,  60759,   6668,  15208,  39090,  59088,  70192,  39089,
        72952,   4705,   4493,   4495,   6026,  10183,  10181,  10180,
        10178,  10195, 114741,  25084,  10184,  10186,  94550,  27145,
        27144,  27147, 112516,  10344,  10345, 125384, 125383,  46488,
       125382,  46491,  46495, 120014, 125361, 125362,  46508,  46505,
        46516,  46518,  46519,  46521,  46523,  46536,  46535,  46534,
        46533], dtype=int64)

Test the shortest path (the weight of the route is the time in minutes from node O to D)

In [58]:
net.shortest_path_length(nodes[0], nodes[1])

86.812

In [63]:
O_lat,O_lon=[43.30460665600362, -2.010031804504358]

In [66]:
O_nodes = net.get_node_ids([O_lon], [O_lat]).values
O_nodes

array([7726], dtype=int64)

In [64]:
D_lat,D_lon=[43.31822992788049, -1.983140736039578]

In [67]:
D_nodes = net.get_node_ids([D_lon], [D_lat]).values
D_nodes

array([118798], dtype=int64)

In [70]:
O_nodes = net.get_node_ids([O_lon], [O_lat]).values
D_nodes = net.get_node_ids([D_lon], [D_lat]).values

# Convert both node arrays to integers (if they are not already)
O_nodes = O_nodes.astype(int)
D_nodes = D_nodes.astype(int)

# Now calculate the shortest path length
net.shortest_path_length(O_nodes[0], D_nodes[0])


22.129